# THIS NOTEBOOK WON'T RUN AS IT USES THE OPENAI API AND PRIVATE KEY.

# 1. IMPORTS

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from datetime import datetime
import openai
import os
import time


In [ ]:
folder = "../temporary/"
df_movies=pd.read_csv(folder + "with_plottone_data.csv",index_col = 'Wikipedia movie ID')

# 2. THEME EXTRACTION

In [5]:
#We decided to use a ChatGPT API which allows us to extract one single theme from each movie summary. The prompt we give to the algorithm to extract the theme is : 
#"Label the movie's theme with a single word based on its plot : ". Then, we clean the results to get rid of all the unwanted characters and store everything in the movie dataframe.

#KEY is secret :-)

def chatgpt(message):
    #time.sleep(0.1)
    completion = openai.chat.completions.create(model="gpt-4o-mini",messages=[{"role": "user","content": message,},],)
    print(completion.choices[0].message.content)
    return completion.choices[0].message.content


In [6]:
#We are limited to 10k api requests per day -> SPLIT to train over 10 days
dfs = np.array_split(df_movies, 10)

df1, df2, df3, df4, df5, df6, df7, df8, df9, df10 = dfs

C:\Users\nicol\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [7]:
#prompt to add before each movie summary
pre = "Label the movie's theme with a single word based on its plot : "
df10['Theme'] = df10['Summary'].apply(lambda x: chatgpt(pre + x) if pd.notna(x) else np.nan)
df10['Theme'] = df10['Theme'].astype('object')


Redemption
Exploitation
Deceit
Resilience
Hope
Monstrosity
"Redemption"
**Deception**
**Friendship**
Afterlife
Belonging
Revolution
Redemption
Integrity
**Ambition**
Redemption
Chaos
**Theme: Redemption**
**Revenge**
**Betrayal**
Adventure
Redemption
**Conflict**
**Escape**
Redemption
Adventure
"Journeys"
Resistance
Adventure
Revenge
Tragedy
Rebellion
**Resistance**
Jealousy
Duty
Betrayal
Isolation
Power
Acceptance
Conflict
**Theme:** Shame
Revenge
Chaos
Betrayal
"Ghosts"
Greed
**Vengeance**
Heroism
Friendship
Perseverance
Inequality
Revenge
**Obsession**
Mystery
Sacrifice
Redemption
Redemption
**Exploitative**
**Horror**
Enlightenment
Partition
Imitation
Pride
**Superiority**
Abuse
Adventure
Love
Transformation
Greed
Redemption
**Despair**
Redemption
Corruption
Survival
Redemption
**Redemption**
Identity
Love
Competition
"Redemption"
Resilience
Love
Family
Sacrifice
**Nostalgia**
**Faith**
Love
Relationships
Compassion
Corruption
Valor
Reunion
Vigilantism
Revenge
Exploitation
Corrupti

# 3. EXPORT TEMPORARY DATASETS

In [14]:
df10.to_csv('../temporary/df10.csv')

# 4. CLEAN THEMES

In [11]:
#import and concat all 10 temporary files
directory = "../temporary"

dataframes = {}

for i in range(1, 11):
    file_path = os.path.join(directory, f"df{i}.csv")
    if os.path.exists(file_path):
        dataframes[f"df{i}"] = pd.read_csv(file_path)

df_movies = pd.concat(dataframes.values(), keys=dataframes.keys(), axis=0)
df_movies.set_index('Wikipedia movie ID', inplace=True)       

In [15]:
df_movies

Movie name  \
Wikipedia movie ID                                                      
975900                                                 Ghosts of Mars   
3196793             Getting Away with Murder: The JonBenét Ramsey ...   
28463795                                                  Brun bitter   
9363483                                              White Of The Eye   
261236                                              A Woman in Flames   
...                                                               ...   
35228177                                     Mermaids: The Body Found   
34980460                                                      Knuckle   
9971909                                             Another Nice Mess   
913762              The Super Dimension Fortress Macross II: Lover...   
12476867                                                      Spliced   

                    Movie release month  Movie release year  \
Wikipedia movie ID                                            
975900                              8.0              2001.0   
3196793                             2.0              2000.0   
28463795                            NaN              1988.0   
9363483                             NaN              1987.0   
261236                              NaN              1983.0   
...                                 ...                 ...   
35228177                            3.0              2011.0   
34980460                            1.0              2011.0   
9971909                             9.0              1972.0   
913762                              5.0              1992.0   
12476867                            NaN              2002.0   

                   Movie release date  Movie box office revenue  \
Wikipedia movie ID                                                
975900                     2001-08-24              2.477118e+07   
3196793                    2000-02-16                       NaN   
28463795                          NaN                       NaN   
9363483                           NaN                       NaN   
261236                            NaN                       NaN   
...                               ...                       ...   
35228177                   2011-03-19                       NaN   
34980460                   2011-01-21                       NaN   
9971909                    1972-09-22                       NaN   
913762                     1992-05-21                       NaN   
12476867                          NaN                       NaN   

                    Movie runtime Movie languages  \
Wikipedia movie ID                                  
975900                       98.0     ['English']   
3196793                      95.0     ['English']   
28463795                     83.0   ['Norwegian']   
9363483                     110.0     ['English']   
261236                      106.0      ['German']   
...                           ...             ...   
35228177                    120.0     ['English']   
34980460                     96.0     ['English']   
9971909                      66.0     ['English']   
913762                      150.0    ['Japanese']   
12476867                     86.0     ['English']   

                                  Movie countries  \
Wikipedia movie ID                                  
975900               ['United States of America']   
3196793              ['United States of America']   
28463795                               ['Norway']   
9363483                        ['United Kingdom']   
261236                                ['Germany']   
...                                           ...   
35228177             ['United States of America']   
34980460            ['Ireland', 'United Kingdom']   
9971909              ['United States of America']   
913762                                  ['Japan']   
12476867                               ['Canada']   

                                     

In [17]:
#verification if index is unique
df_movies.index.is_unique

True

In [34]:
#clean the Theme column removing unwanted caracters and bad answers

df_movies['Theme'] = df_movies['Theme'].str.replace(r'["*:.]', '', regex=True)
df_movies['Theme'] = df_movies['Theme'].str.lstrip('theme ')
df_movies['Theme'] = df_movies['Theme'].str.lstrip('Theme ')
df_movies['Theme'] = df_movies['Theme'].str.lstrip('theme of the movie can be labeled as ')
df_movies['Theme'] = df_movies['Theme'].apply(lambda x: np.nan if isinstance(x, str) and len(x) > 20 else x)

In [21]:
#check the most popular themes
df_movies['Theme'].value_counts()

Theme
Redemption       4832
Love             2515
Betrayal         2389
Revenge          2315
Survival         1742
                 ... 
Culinary            1
Language            1
Peer Pressure       1
Sharing             1
Ignorance           1
Name: count, Length: 1467, dtype: int64

# 5. EXPORT

In [40]:
#df_movies.to_csv("../generated/final_movie_dataset.csv")

# 6. ADDITIONNAL THEME CORRECTIONS

In [6]:
final_movie_dataset = pd.read_csv("../generated/final_movie_dataset.csv")

# Make all themes lowercase
final_movie_dataset['Theme'] = final_movie_dataset['Theme'].str.lower()

# Additionnal theme corrections for missing 't' at the beginning of the word + merging of similar themes
corrections = { 
    "ransformation": "transformation",
    "ransition": "transition",
    "ptation": "temptation",
    "ragedy": "tragedy",
    "ranscendence": "transcendence",
    "radition": "tradition",
    "ransport": "transport",
    "ransgression": "transgression",
    "ransmutation": "transmutation",
    "ransparency": "transparency",
    "ransportation": "transportation",
    "ransmigration": "transmigration",
    "rture": "torture",
    "rritory": "territory",
    "rafficking": "trafficking",
    "vriendschap": "friendship",
    "vengeance": "revenge" #grouping for analysis
}

# Function to correct themes
def correct_theme(theme):
    if pd.isna(theme):  # Handle NaN values
        return theme
    return corrections.get(theme, theme)  # Apply correction if it exists

# Apply the correction to the 'Theme' column
final_movie_dataset['Corrected_Theme'] = final_movie_dataset['Theme'].apply(correct_theme)

# Drop 'Theme' column and change 'Corrected_Theme' column to 'Theme'
final_movie_dataset = final_movie_dataset.drop(['Theme'], axis=1)
final_movie_dataset.rename({'Corrected_Theme': 'Theme'}, axis=1, inplace=True)

# 

In [8]:
final_movie_dataset.to_csv("../generated/final_movie_dataset.csv")